# CSC311 Lab 10: Clustering via Bernoulli Mixture Models

In this lab, we will build a Bernoulli Mixture Model to cluster
movie reviews. We will use the same movie reviews data set as in Lab 9.
However, we will *not* use the positive/negative labels. Instead,
we will analyze the reviews themselves to see whether there are
interesting structure in the reviews, in the form of clusters.

By the end of this lab , you will be able to:

1. Explain the connection between the Bernoulli Mixture Model and the Naive Bayes model from week 8.
2. Explain the connection between the Bernoulli Mixture Model and the Gaussian Mixture model from lecture 10.
3. Apply the E-M algorithm to fit a BMM model to perform clustering over Bernoulli variables.
4. Track the progress of the E-M algorithm by tracking the MLE.
5. Analyze the results of the clustering by manually observing the clusters.

Please work in groups of 1-2 during the lab.

## Submission

If you are working with a partner, start by creating a group on Markus.
If you are working alone,
click "Working Alone".

Submit the ipynb file `lab10.ipynb` on Markus
**containing all your solutions to the Graded Task**s.
Your notebook file must contain your code **and outputs** where applicable,
including printed lines and images.
Your TA will not run your code for the purpose of grading.

For this lab, you should submit the following:


- Part 2. your implementation of the `e_step` function (3 points).
- Part 2. your implementation of the `m_step` function (3 points).
- Part 2. your implementation of the `em_bmm` function (1 points).
- Part 3. your code and interpretation of the results of running `em_bmm` on all data, with 2 clusters (2 points)
- Part 3. your code and interpretation of the results of running `em_bmm` on positive reviews, with 5 clusters (1 points)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

## Acknowledgements

Data is a variation of the one from https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews,
pre-processed so that only 1000 words are in the training/test set.

## Part 1. Data

We will use the same movie review data set as in lab 9.

In [ ]:
# Download tutorial data files.
!wget https://www.cs.toronto.edu/~lczhang/311/lab09/trainvalid.csv
!wget https://www.cs.toronto.edu/~lczhang/311/lab09/test.csv

--2025-03-23 00:16:19--  https://www.cs.toronto.edu/~lczhang/311/lab09/trainvalid.csv
Resolving www.cs.toronto.edu (www.cs.toronto.edu)... 128.100.3.30
Connecting to www.cs.toronto.edu (www.cs.toronto.edu)|128.100.3.30|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1464806 (1.4M) [text/csv]
Saving to: ‘trainvalid.csv’

trainvalid.csv      100%[===================>]   1.40M  5.59MB/s    in 0.3s    

2025-03-23 00:16:19 (5.59 MB/s) - ‘trainvalid.csv’ saved [1464806/1464806]

--2025-03-23 00:16:19--  https://www.cs.toronto.edu/~lczhang/311/lab09/test.csv
Resolving www.cs.toronto.edu (www.cs.toronto.edu)... 128.100.3.30
Connecting to www.cs.toronto.edu (www.cs.toronto.edu)|128.100.3.30|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 491538 (480K) [text/csv]
Saving to: ‘test.csv’

test.csv            100%[===================>] 480.02K  2.59MB/s    in 0.2s    

2025-03-23 00:16:20 (2.59 MB/s) - ‘test.csv’ saved [491538/491538]



In [ ]:
import csv

# Training/Validation set
trainfile = "trainvalid.csv"
data = list(csv.reader(open(trainfile)))

# Build the vocabulary
vocab = set()
for review, label in csv.reader(open(trainfile)):
    words = review.split()
    for w in words:
        vocab.add(w)
vocab = list(vocab) # len(vocab) == 1000

**Task**: Use the function `make_bow` that you wrote in lab 9 to create a
data matrix consisting of bag-of-word features. We will then create three
matrices: one for all reviews, one for only positive reviews, and one
for only negative reviews.

In [ ]:
def make_bow(data, vocab):
    """
    Produce the bag-of-word representation of the data, along with a vector
    of labels. You *may* use loops to iterate over `data`. However, your code
    should not take more than O(len(data) * len(vocab)) to run.

    Parameters:
        `data`: a list of `(review, label)` pairs, like those produced from
                `list(csv.reader(open("trainvalid.csv")))`
        `vocab`: a list consisting of all unique words in the vocabulary

    Returns:
        `X`: A data matrix of bag-of-word features. This data matrix should be
             a numpy array with shape [len(data), len(vocab)].
             Moreover, `X[i,j] == 1` if the review in `data[i]` contains the
             word `vocab[j]`, and `X[i,j] == 0` otherwise.
        `t`: A numpy array of shape [len(data)], with `t[i] == 1` if
             `data[i]` is a positive review, and `t[i] == 0` otherwise.
    """
    X = np.zeros([len(data), len(vocab)])
    t = np.zeros([len(data)])

    # TODO: fill in the appropriate values of X and t --- DONE ---
    for i, (review, label) in enumerate(data):
        words = review.split()
        for w in words:
            if w in vocab:
                X[i, vocab.index(w)] = 1
        t[i] = 1 if label == 'positive' else 0

    return X, t

# Data matrix with all reviews in trainvalid.csv
X_all, t_all = make_bow(data, vocab)

# Data matrix with only positive reviews
data_pos = [data[i] for i in np.where(t_all == 1)[0]] # actual review text
X_pos = X_all[t_all == 1]                             # data matrix

# Data matrix with only negative reviews
data_neg = [data[i] for i in np.where(t_all == 0)[0]] # actual review text
X_neg = X_all[t_all == 0]                             # data matrix

## Part 2. Bernoulli Mixture Model

A Bernoulli Mixture Model is a finite mixture of random binary, independent features.
$$p({\bf x})=  \sum_{k=1}^K p({\bf x} | z=k) \ P(z=k) = (\sum_{k=1}^K \prod_{j=1}^D p(x_j | z=k) ) P(z=k)$$
Where $p(z)$ describes the distribution of a categorical random variable,
and each $p(x_j | z=k)$ describes the distribution of a Bernoulli random variable.
The parameters of our model include $\pi_k=P(z=k)$ (if there are $K$ classes,
then we have parameters $\pi_1$, $\pi_2$, \dots, $\pi_{K-1}$), and
$\theta_{jk}=p(x_j=1 |z=k)$ (of which there are $DK$ in total).

Note the similarities and differences between a BMM v.s.
a mixture of Gaussians described in class (written MoG, also Gaussian Mixture Model or GMM).
In both mixture models, we have
$$p({\bf x})= p({\bf x} | z=k) \ P(z=k)$$,
with $P(z)$ being a categorical distribution. The difference is that in
a mixture of Gaussians, $p({\bf x} | z=k)$ is a Gaussian, whereas  in a
Bernoulli mixture model, $p({\bf x} | z=k) = \prod_{j=1}^D p(x_j | z=k)$.
That is, each $p({\bf x} | z=k)$ consists of $D$ different conditionally
independent Bernoullis $p(x_j | z=k)$, where ${\bf x} \in \mathbb{R}^D$.

The decomposition of
$p({\bf x})= \sum_{k=1}^K (\prod_{j=1}^D p(x_j | z=k) ) P(z=k)$
is similar to Naive Bayes. However,
unlike in the Naive Bayes lab, our random variable $z$ is **not observed**. Thus,
$z$ is called a **latent variable**.

<!--
Still, we can marginalize out $z$ to write
down the likelihood of our data:
\begin{align*}
\ell(\{\pi_k, \theta_{jk}\})
    &= \sum_{k=1}^K \sum_{i=1}^N \log\Big\{ p({\bf x}^{(i)}|z^{(i)})p(z^{(i)}=k)\Big\}\\
    &= \sum_{k=1}^K \sum_{i=1}^N \log \Big\{p(z^{(i)}=k) \prod_{j=1}^D p(x_j^{(i)} | z^{(i)}=k) \Big\}\\
    &= \sum_{k=1}^K \sum_{i=1}^N \left[ \log p(z^{(i)}=k) + \sum_{j=1}^D \log p(x_j^{(i)} | z^{(i)}=k) \right] \\
    &= \sum_{k=1}^K \sum_{i=1}^N \log p(z^{(i)}=k) + \sum_{k=1}^K \sum_{j=1}^D \sum_{i=1}^N \log p(x_j^{(i)} | z^{(i)}=k)
\end{align*}
-->

In this section, we will fit the parameters of this BMM model to maximize
the likelihood of our movie review data.

Because $z$ is latent, we will need to use the Expectation-Maximization (E-M) algorithm.
EM is an optimization approach where we alternatively optimize the
responsibilities $r_k^{(i)}= P(z^{(i)}=k | {\bf x}^{(i)}; \theta, \pi)$ given the parameters,
and optimize the parameters given the cluster responsibilities $r_k^{(i)}$ for each data point $i$
and cluster $k$.

**Graded Task**: The "E" step of the Expectation-Maximization Algorithm
assign the **responsibility** $r_k^{(i)}$ of component $k$ for data point $i$ using the posterior probability:
$$r_k^{(i)}= P(z^{(i)}=k | {\bf x}^{(i)}; \theta, \pi)$$
Complete the function `e_step` which performs this computation.
You **may** use loops, but minimizing the use of loops would help make your code run faster.

If you are not sure where to start, a good place is to review the "E" step for
the Gaussian mixture model. Computation, the "E" step for the GMM is the *inference*
stage of a Gaussian Discriminate Analysis model. Likewise, the "E" step of our
BMM model is computationally the same as the *inference* stage of a Naive Bayes
model. If this analogy doesn't makes sense, please ask!
Once this analogy makes sense, it should be clear what computation needs to be
done, and how this math relates to your work from lab 9.

In [ ]:
def e_step(X, pi, theta):
    """
    Perform the "E" step of the E-M Algorithm for a BMM.
    In other words, given the data matrix `X` and estimates to the
    parameters `theta` and `pi`, estimate $P(z=k|{\bf x})$ for each
    ${\bf x}$ in the data matrix.

    Parameters:
        `X` - a data matrix of bag-of-word features of shape [N, D],
              where N is the number of data points and D is the vocabulary size.
              X[i,j] should be either 0 or 1. Produced by the make_bow() function.
        `pi` - a vector of shape [K], where `pi[k]` corresponds to
              an estimate of the parameter $\pi_{k} = P(z=k)$.
              Precondition: `np.sum(pi) = 1` so that the $\pi_k$s describe a
              probability distribution.
        `theta` - a matrix of shape [D, K], where `theta[j, k]` corresponds to
              an estimate of the parameter $\theta_{jk} = P(x_j = 1 | z=k)$

    Returns:
        `R` - a matrix of shape [N, K], where `R[j, k]` corresponds to the
              value $P(z^{(j)}=k| {\bf x^{(j)}})$ computed using the estimated
              parameters `theta` and `pi`.
              Each row of `R` should sum up to 1, i.e. `sum(R[j,:]) == 1`.
    """
    N, D = X.shape
    D, K = theta.shape
    R = np.zeros([N, K])

    # TODO: fill in the elements of R
    # Remember the log trick that we used to avoid computing a product
    # of small numbers, from lab 9.

    # --- DONE ---
    for n in range(N):
        for k in range(K):
            # Initialize with log(pi_k)
            log_prob = np.log(pi[k])

            # Add log probabilities for each feature
            for d in range(D):
                if X[n, d] == 1:
                    log_prob += np.log(theta[d, k])
                else:
                    log_prob += np.log(1 - theta[d, k])

            # Store the unnormalized log probability
            R[n, k] = log_prob

        # Convert from log probabilities to actual probabilities
        # Subtract max for numerical stability before exp
        log_max = np.max(R[n])
        R[n] = np.exp(R[n] - log_max)

        # Normalize to get valid probabilities that sum to 1
        R[n] = R[n] / np.sum(R[n])

    return R

# Test example with N=2 movie reviews, D=1 words, K=2 clusters
X_basic_check = np.array([[1], [0]]) # first data point has the word present
                                     # second data point has the word absent
pi_basic_check = np.array([0.5, 0.5]) # both clusters have equal probability
theta_basic_check = np.array([[0.9 ,  # first cluster has P(word present|cluster) = 0.9
                               0.2]]) # second cluster has P(word present|cluster) = 0.2

# Please include the output of the below call in your submission.
# Think: what do the 4 results numbers here mean? You should
# see that the first data point has a higher probability of
# being in the first cluster (vs the second cluster), and vice-versa
# for the second data point. Why would this make sense?
# (You do not need to write a response. The question is here to
# help you interpret what this function is doing.)
e_step(X_basic_check, pi_basic_check, theta_basic_check)

array([[0.81818182, 0.18181818],
       [0.11111111, 0.88888889]])

**Graded Task**: The "M" step of the Expectation-Maximization Algorithm
computes estimates of $\pi$ and $\theta_{jk}$ via maximum likelihood,
where each the parameters for each class $k$ is fit with a weighted dataset.
The weights are proportional to the responsibilities.
Complete the function `m_step` which performs this computation.
You **may** use loops, but minimizing the use of loops would help make your code run faster.

If you are not sure where to start, a good place is to review the "M" step for
the Gaussian mixture model. Computation, the "M" step for the GMM is
a slight variation of the *learning* stage of a Gaussian Discriminate Analysis model.
Likewise, the "E" step of our BMM model is computationally very similar as the
*learning* stage of a Naive Bayes model (except that $r_j^{(i)}$ is now continuous
rather than being exactly 0 or 1).
Once again, if this analogy doesn't makes sense, please ask!
And when this analogy makes sense, it should be clear what computation
needs to be done, and how this math relates to your work from lab 9.

In [ ]:
def m_step(X, R):
    """
    Perform for a BMM the "M" step of the E-M Algorithm for a BMM.
    In other words, given the data matrix `X` and estimates to the
    parameters `theta` and `pi`, estimate $P(z=k|{\bf x})$ for each ${\bf x}$ in the
    data matrix.

    Parameters:
        `X` - a data matrix of bag-of-word features of shape [N, D],
              where N is the number of data points and D is the vocabulary size.
              X[i,j] should be either 0 or 1. Produced by the make_bow() function.
        `R` - a matrix responsibilities of shape [N, K], where `R[j, k]` corresponds
              to the value $P(z^{(j)}=k| {\bf x^{(j)}})$ computed during the e_step.
              Precondition: Each row of `R` sums to 1, i.e. `sum(R[j,:]) == 1`

    Returns:
        `theta` - a matrix of shape [D, K], where `theta[j, k]` corresponds to
              the MLE estimate of the parameter $\theta_{jk} = p(x_j = 1 | z=k)$
        `pi` - a vector of shape [K], where `pi[k]` corresponds to
              the MLE estimate of the parameter $\pi_{k} = P(z=k)$.
              We should have `np.sum(pi) = 1` so that the $\pi_k$s describe a
              probability distribution.
    """
    N, D = X.shape
    N, K = R.shape

    # Calculate pi (cluster probabilities) --- DONE ---
    # pi_k = sum of responsibilities for cluster k / total number of data points
    pi = np.sum(R, axis=0) / N

    # Calculate theta (word probabilities given cluster)
    theta = np.zeros([D, K]) # TODO: fill this!

     # For each feature and cluster
    for d in range(D):
        for k in range(K):
            # Weighted count of word occurrences in cluster k
            numerator = np.sum(X[:, d] * R[:, k])
            # Sum of responsibilities for cluster k
            denominator = np.sum(R[:, k])
            # Probability of word d being present in cluster k
            theta[d, k] = numerator / denominator

    return (pi, theta)

# Test example with N=2 movie reviews, D=1 words, K=2 clusters
X_basic_check = np.array([[1], [0]]) # first data point has the word present
                                     # second data point has the word absent
R_basic_check = np.array([[0.7, 0.3],
                          [0.5, 0.5]])

# Please include the output of the below call in your submission.
# Think: you should see that the "pi"s are such that the first
# cluster is a tad more likely to be observed. Why does this
# make sense? You should also see that the estimate for
# P(word present|cluster 1) is higher than P(word present|cluster 2).
# Why does this also make sense? (You do not need to write a response.
# The question is here to help you interpret what this function is doing.)
m_step(X_basic_check, R_basic_check)

(array([0.6, 0.4]), array([[0.58333333, 0.375     ]]))

**Graded Task**: Complete the function `em_bmm` by calling the functions `e_step`
and `m_step` that you wrote previously.

In [ ]:
def em_bmm(X, K=5, num_iter=10):
    """
    Use the E-M algorithm to compute parameters `theta` and `pi` of a
    Bernouli Mixture Model fit on the data `X`.

    Parameters:
        `X` - a data matrix of bag-of-word features of shape [N, D],
              where N is the number of data points and D is the vocabulary size.
              X[i,j] should be either 0 or 1. Produced by the make_bow() function.
        `K` - number of classes (i.e. number of Bernouli Mixtures in the model)
        `num_iter` - number of iterations to run the E-M algorithm

    Returns:
        `theta` - a matrix of shape [D, K], where `theta[j, k]` corresponds to
              the E-M estimate of the parameter $\theta_{jk} = P(x_j = 1 | z=k)$
        `pi` - a vector of shape [K], where `pi[k]` corresponds to
              the E-M estimate of the parameter $\pi_{k} = P(z=k)$.
              We should have `np.sum(pi) = 1` so that the $\pi_k$s describe a
              probability distribution.
    """
    N, vocab_size = X.shape

    pi = np.ones([K]) * (1/K)  # start with equal class probabilities
    theta = np.random.rand(vocab_size, K) / 10 # initialize p(x_j|z) to be between 0 and 0.1

    for j in range(num_iter):
        # e-step
        # TODO - what needs to be computed here?
        # Calculate responsibilities (posterior probabilities of each data point belonging to each cluster)
        R = e_step(X, pi, theta)

        # m-step
        # TODO - what needs to be computed here?
        # Update parameters based on the calculated responsibilities
        pi, theta = m_step(X, R)

    return pi, theta

## Part 3. Clustering

In this section, we will use the `em_bmm` to cluster our movie reviews. We will
*not* use the positive/negative reviews as labels, and the clusters that we find
may or may not correspond to easily interpretable results.

**Graded Task**: Fit a BMM model using the EM algorithm on the entire data (`X_all`).
Use `K=2` and `num_iter=10` to cluster the data into 2 classes.
Then, use `e_step` to compute the class responsibilities `Z`, for each data point.
After you have computed `Z`, use the below code to print out some example reviews
in each of the two classes. Do these classes correspond to positive/negative reviews?
If not, how would you describe the difference between the two clusters?

In [ ]:
pi, theta = em_bmm(X_all, K=2, num_iter=10) # TODO
Z = e_step(X_all, pi, theta) # TODO

classes = np.argmax(Z, axis=1)
for k in range(Z.shape[1]):
    print("======= Samples From Class", k, "=========")
    for i in np.where(classes == k)[0][:10]:
        print(data[i][0])

# TODO: Write your interpretation of the classes here
# The reviews in the classes don't necessarily represent
# positive or poor reviews. Class 0 seems to focus on
# classics that are reviewed post release in a modern
# context. Class 1 seems to contain foreign series,
# /niche content that is not mainstream.


======= Samples From Class 0 =========
this movie has some things that are pretty amazing first it is supposed to be based on a true story the problem was that by then you would be able to hear it
lets start with the good things this is not a movie for kids the action scenes are great here too ok over to the things i didnt like thats that
i enjoyed this movie as a kid when it came out and to this day still do i am surprised not too many people know about it cool flick
i love this film it is well written and acted and has good cinematography see it buy it show it to your friends it still made me happy when everything worked out well in the end
ever since i was a child this has been one of my favorite stories i want nothing from you why cant we be friends it is a moving performance and one of the movies most dramatic scenes not
you want a friend get a dog at first all good no bad right everything goes to hell of course love that guy on some level i like that take the kids
but the plot i 

**Graded Task**: Fit a BMM model using the EM algorithm on only the positive
reviews (`X_pos`).
Use `K=5` and `num_iter=10` to cluster the data into 5 classes.
Then, use `e_step` to compute the class responsibilities `Z`, for each data point.
After you have computed `Z`, use the below code to print out some example reviews
in each of the five classes.

Provide an interpretation for what these classes mean.

In [19]:
pi, theta = em_bmm(X_pos, K=5, num_iter=10) # TODO
Z = e_step(X_pos, pi, theta) # TODO

classes = np.argmax(Z, axis=1)
for k in range(Z.shape[1]):
    print("======= Samples From Class", k, "=========")
    for i in np.where(classes == k)[0][:10]:
        print(data_pos[i][0])

# TODO: Write your interpretation of the classes here

# Class zero seems to again be the justification for the \
# movies niche nature. Positive in the sense of recognzing
# what it set out to do.
# Class one is a review focused on the positive qualities
# of the film. It is full of positive adjectives which
# detail the film's strengths.
# Class two seems to be a review focused on the enjoyment
# factor of the movie as a form of entertainment.
# Class three seems to focus on the emotional impact
# of the film as a contemplative work of art, as
# well as the themes it tackles and how the audience
# can relate to the characters.
# Class four seems to deal with an indie horror/thriller
# film possibly. The reception is lukewarm but, the
# reviews understand that it is not the team's best work.

<ipython-input-15-7c0444310987>:42: RuntimeWarning: divide by zero encountered in log
  log_prob += np.log(theta[d, k])


======= Samples From Class 0 =========
first of all i want to point on screen play
things come to a head and one has to keep watching to follow up on such a sequence
much to my surprise it wasnt about that at all
he has no friends and all he wants to do is write and have someone like his writing
i kept expecting it to fall apart but it never really did
you know how its going to end but it has a great time getting there
i dont know why its horrible what they did to the film
but that scene will stay with you go for it
he gets a chance to show off a bit i thought it was a good movie hes just hilarious you could do much much worse
its not even close we need something different s s s version does have a better soundtrack than the original version s version
======= Samples From Class 1 =========
i would love to see either of them in another movie
this movie is great if you enjoy watching b class movies that is
if you can do that it really is good s s s s i doubt it too bad
but it is still do

**Task**: Fit a BMM model using the EM algorithm on only the negative
reviews (`X_neg`).
Use `K=5` and `num_iter=10` to cluster the data into 5 classes.
Then, use `e_step` to compute the class responsibilities `Z`, for each data point.
After you have computed `Z`, use the below code to print out some example reviews
in each of the five classes.

Provide an interpretation for what these classes mean.

In [20]:
pi, theta = em_bmm(X_neg, K=5, num_iter=10) # TODO
Z = e_step(X_neg, pi, theta) # TODO

classes = np.argmax(Z, axis=1)
for k in range(Z.shape[1]):
    print("======= Samples From Class", k, "=========")
    for i in np.where(classes == k)[0][:10]:
        print(data_neg[i][0])

# TODO: Write your interpretation of the classes here

# Class zero seems to focus on the film's lack of
# intrigue, and lost potential.

# Class one seems to focus on the film's low-effort
# nature. The cast seems disinterested and the plot
# is innefective.

# Class two focuses on the poor casting/script

# Class three repeats that the movie is a waste
# of time, and among the worst the audience has seen

# Class four broadly covers how non-sensical
# the actions of the cast are, and the poor composition
# of the scenes/cinematography.

======= Samples From Class 0 =========
not good and not bad either hes a very good actor and should probably be an action star or something why do they make movies like this in the first place
after all movies for free i told myself well hes done the good and the bad as far as films and tv go when they finally take off the effects are truly horrible but the horror doesnt stop there
this has got to be one of the worst movies ive ever seen do they laugh when they create all this ridiculous stuff or do they actually think theyre doing something interesting i wonder its the best thing to do god
all the way though i was thinking to myself oh god why why does going into the house make her come after you it doesnt make sense you can see everything coming which just left you feeling that there was no point in watching oh shes behind her
the reviews i read for this movie were pretty decent so i decided to check it out bad idea and i didnt care about any of them perhaps a better writer could hav